In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import joblib, json, sys, os
from IPython.display import Image, display_png
import matplotlib.pyplot as plt
import pandas as pd

import sys, os, os.path, re, csv, math
import linecache
import numpy as  np
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean,stdev
import seaborn as sns

import csv
import pprint
import ants
import tifffile
from multiprocessing import Pool
import datetime
from itertools import product

import numpy as np
import tifffile
import os
import pickle
import joblib
import pandas as pd
from matplotlib import pyplot as plt
import concurrent.futures
import gc
import json

import datetime

In [2]:
conditions = {

    "WT_1m": ["#4_APPmodel_Abeta_Ctr1m_1","#4_APPmodel_Abeta_Ctr1m_2","#4_APPmodel_Abeta_Ctr1m_3", "#4_APPmodel_Abeta_Ctr1m_4"],
 
    "WT_3m": ["#4_APPmodel_Abeta_Ctr3m_1","#4_APPmodel_Abeta_Ctr3m_2","#4_APPmodel_Abeta_Ctr3m_3", "#4_APPmodel_Abeta_Ctr3m_4",],
    "WT_5m": ["#4_APPmodel_Abeta_Ctr5m_1","#4_APPmodel_Abeta_Ctr5m_2","#4_APPmodel_Abeta_Ctr5m_3", "#4_APPmodel_Abeta_Ctr5m_4"],
    "WT_7m": ["#4_APPmodel_Abeta_Ctr7m_1","#4_APPmodel_Abeta_Ctr7m_2", "#4_APPmodel_Abeta_Ctr7m_3", "#4_APPmodel_Abeta_Ctr7m_4"],
    "WT_9m": ["#4_APPmodel_Abeta_Ctr9m_1","#4_APPmodel_Abeta_Ctr9m_2","#4_APPmodel_Abeta_Ctr9m_3", "#4_APPmodel_Abeta_Ctr9m_4"],

    "APP_1m": ["#4_APPmodel_Abeta_APP1m_1","#4_APPmodel_Abeta_APP1m_2","#4_APPmodel_Abeta_APP1m_3","#4_APPmodel_Abeta_APP1m_4"],
    "APP_3m": ["#4_APPmodel_Abeta_APP3m_1","#4_APPmodel_Abeta_APP3m_2", "#4_APPmodel_Abeta_APP3m_3", "#4_APPmodel_Abeta_APP3m_4"],
   "APP_5m": ["#4_APPmodel_Abeta_APP5m_1","#4_APPmodel_Abeta_APP5m_2","#4_APPmodel_Abeta_APP5m_3","#4_APPmodel_Abeta_APP5m_4"],
    "APP_7m": ["#4_APPmodel_Abeta_APP7m_1","#4_APPmodel_Abeta_APP7m_2", "#4_APPmodel_Abeta_APP7m_3", "#4_APPmodel_Abeta_APP7m_4"],
    "APP_9m": ["#4_APPmodel_Abeta_APP9m_1","#4_APPmodel_Abeta_APP9m_2","#4_APPmodel_Abeta_APP9m_3","#4_APPmodel_Abeta_APP9m_4"],

}

In [3]:
Samples = [path for paths in conditions.values() for path in paths]
Samples

['#4_APPmodel_Abeta_Ctr1m_1',
 '#4_APPmodel_Abeta_Ctr3m_1',
 '#4_APPmodel_Abeta_Ctr5m_1',
 '#4_APPmodel_Abeta_Ctr7m_1',
 '#4_APPmodel_Abeta_Ctr9m_1',
 '#4_APPmodel_Abeta_APP1m_1',
 '#4_APPmodel_Abeta_APP3m_1',
 '#4_APPmodel_Abeta_APP5m_1',
 '#4_APPmodel_Abeta_APP7m_1',
 '#4_APPmodel_Abeta_APP9m_1']

In [5]:
# Map to the Raw Neuron Atlas (CUBIC-R+ Space) Using ANTs
# Next, map the data to the Neuron Atlas.
# Abeta Sample Assumes 10x10x10 μm Scale Voxels

In [9]:
import json, os.path, os, re, time
import tifffile
import joblib
from docopt import docopt
import subprocess as sp
import pandas as pd
import numpy as np
import nibabel as nib
import scipy.spatial
import shutil

dt_annotated = np.dtype([
    ('X(um)', 'f4'), ('Y(um)', 'f4'), ('Z(um)', 'f4'),
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
])

def register(atlas_basedir, merging_basedir, mapping_basedir,
             prefix_ANTs, atlas_voxel_unit, sample_base_name, num_cpus=36,
             atlas_basename="iso_50um"):
    atlas_tif_path = os.path.join(atlas_basedir, "{}.tif".format(atlas_basename))
    atlas_nii_path = os.path.join(atlas_basedir, "{}.nii.gz".format(atlas_basename))
    #moving_tif_path = os.path.join(merging_basedir, "whole.tif")
    moving_tif_path = os.path.join(merging_basedir, "BOBO-1_50um_resized.tif")
    moving_nii_path = os.path.join(mapping_basedir, "whole.nii.gz")

    # prepare nifti image for atlas
    print("[*] Preparing nifti image for atlas...")
    img_atlas = tifffile.imread(atlas_tif_path)
    if not os.path.exists(atlas_nii_path):
        nii_atlas = nib.Nifti1Image(np.swapaxes(img_atlas,0,2), affine=None)
        aff = np.diag([-atlas_voxel_unit,-atlas_voxel_unit,atlas_voxel_unit,1])
        nii_atlas.header.set_qform(aff, code=2)
        nii_atlas.to_filename(atlas_nii_path)
    # prepare nifti image for moving
    print("[*] Preparing nifti image for moving...")
    img_moving = tifffile.imread(moving_tif_path)
    nii_moving = nib.Nifti1Image(np.swapaxes(img_moving,0,2), affine=None)
    aff = np.diag([-atlas_voxel_unit,-atlas_voxel_unit,atlas_voxel_unit,1])
    nii_moving.header.set_qform(aff, code=2)
    nii_moving.to_filename(moving_nii_path)

    print("start ANTs")
    #start_time = datetime.datetime.now()
    #print("ANTs Start time:", start_time)
    atlas_img = ants.image_read(atlas_nii_path)
    src_img = ants.image_read(moving_nii_path)
    out = ants.registration(fixed=atlas_img , moving=src_img, type_of_transform='SyNCC' )
    warped = out['warpedmovout']
    tifffile.imwrite(os.path.join(mapping_basedir, "F2M_Warped.tif"), ((warped.numpy().T).astype(np.float32)).astype(np.float32))
    
    for transform in out['invtransforms']:

        extension = transform.split('/')[-1].split('.')[-1]
        if extension != 'gz': # .matである。
            shutil.copy(transform, os.path.join(mapping_basedir, "F2M_0GenericAffine_inv.mat"))

        if extension == 'gz':  # .nii.gz の場合
            extension = 'nii.gz'
            shutil.copy(transform, os.path.join(mapping_basedir, "F2M_InvWarped.nii.gz"))

    for transform in out['fwdtransforms']:

        extension = transform.split('/')[-1].split('.')[-1]
        if extension != 'gz': # .matである。
            shutil.copy(transform, os.path.join(mapping_basedir, "F2M_0GenericAffine.mat"))

        if extension == 'gz':  # .nii.gz の場合
            extension = 'nii.gz'
            shutil.copy(transform, os.path.join(mapping_basedir, "F2M_InvWarped_fwd.nii.gz"))

    return


def map_and_annotate_cellstacks(src_csv_path, annotated_csv_path, 
                                prefix_ANTs, 
                                downscale_unit):
    

    df_src = pd.read_csv(src_csv_path + "Abeta_positive_voxel_coordinates.csv")
    
    # um化
    df = pd.DataFrame({
        "x":df_src["X"].astype(np.float32)*downscale_unit,
        "y":df_src["Y"].astype(np.float32)*downscale_unit,
        "z":df_src["Z"].astype(np.float32)*downscale_unit,
    })
    
    df_transformed = ants.apply_transforms_to_points(
    dim=3,
    points=df,
    transformlist=[os.path.join(src_csv_path, "F2M_0GenericAffine.mat"),
                os.path.join(src_csv_path, "F2M_InvWarped.nii.gz")],
    whichtoinvert = [True, False]
    )
    
    data_annotated =  np.zeros((len(df_transformed),), dtype=dt_annotated)
    
    data_annotated["X(um)"] = df_src["X"].astype(np.float32)
    data_annotated["Y(um)"] = df_src["Y"].astype(np.float32)
    data_annotated["Z(um)"] = df_src["Z"].astype(np.float32)
    
    data_annotated["mapped_x"] = df_transformed["x"].astype(np.float32)
    data_annotated["mapped_y"] = df_transformed["y"].astype(np.float32)
    data_annotated["mapped_z"] = df_transformed["z"].astype(np.float32)


    data_annotated.tofile(src_csv_path + "Transformed_to_R_atlas_space_Abeta_positive_voxel_coordinates.bin")


for Sample in Samples:
    print(Sample)
    start_time = datetime.datetime.now()
    print("Start time:", start_time)

    try:
        if 1==1:
            # Extract Voxel Coordinates with True (1) Values from image_tau_thresh.tif    
            image_Aeta_extracted = tifffile.imread(f"/export3/Imaging/ds4_Data5/Abeta/{Sample}/image_tau_thresh.tif")

            # The shape of image_Aeta_extracted is in the order (z, y, x).
            # Extract xyz coordinates of voxels with a value of 1 and save them as a CSV file.

            z_indices, y_indices, x_indices = np.where(image_Aeta_extracted == 1)

            coordinates_df = pd.DataFrame({
                'X': x_indices,     
                'Y': y_indices,
                'Z': z_indices
            })

            # Save as CSV
            coordinates_df.to_csv(f"/export3/Imaging/Abeta/{Sample}/Abeta_positive_voxel_coordinates.csv", index=False)    
            end_time = datetime.datetime.now()
            print("CSV save End time:", end_time)
            print("Duration:", end_time - start_time)

        if 1==1:
            
            # Register Subsampled 50 μm Nucleus-Stained Image to the CUBIC-R+  (50 μm Cell Nuclear Density with Ventricle)
            register("/export3/Imaging/Axial/Neurorology/8w_B6JWT_atlas_1_2022_0924_1847",
                     f"/export3/Imaging/Abeta/{Sample}/",
                     f"/export3/Imaging/Abeta/{Sample}/", 
                     "/usr/local/ants-2.1.0-redhat/",
                     50.0,
                     "BOBO-1_50um_resized",
                     40,
                     "iso_50um_R_ver4_nuclear_with_ventricle",  
                    ) 
            end_time = datetime.datetime.now()
            print("ANTs End time:", end_time)
            print("Duration:", end_time - start_time)


        # Point transfer
        if 1==1:
            map_and_annotate_cellstacks(
                f"/export3/Imaging/Abeta/{Sample}/",
                f"/export3/Imaging/Abeta/{Sample}/",
                "/usr/local/ants-2.1.0-redhat/",
                downscale_unit = 10.0,
            )
            end_time = datetime.datetime.now()
            print("Point transfer End time:", end_time)
            print("Duration:", end_time - start_time)   
    except:
        print("skip")

#4_APPmodel_Abeta_Ctr1m_1
Start time: 2024-07-11 11:08:43.113441
CSV save End time: 2024-07-11 11:09:28.130959
Duration: 0:00:45.017518
[*] Preparing nifti image for atlas...
[*] Preparing nifti image for moving...
start ANTs
ANTs End time: 2024-07-11 11:46:42.416259
Duration: 0:37:59.302818
Point transfer End time: 2024-07-11 11:46:45.956721
Duration: 0:38:02.843280
#4_APPmodel_Abeta_Ctr3m_1
Start time: 2024-07-11 11:46:45.956978
CSV save End time: 2024-07-11 11:47:32.820040
Duration: 0:00:46.863062
[*] Preparing nifti image for atlas...
[*] Preparing nifti image for moving...
start ANTs
ANTs End time: 2024-07-11 12:46:09.085892
Duration: 0:59:23.128914
Point transfer End time: 2024-07-11 12:46:15.436255
Duration: 0:59:29.479277
#4_APPmodel_Abeta_Ctr5m_1
Start time: 2024-07-11 12:46:15.438424
CSV save End time: 2024-07-11 12:46:57.102600
Duration: 0:00:41.664176
[*] Preparing nifti image for atlas...
[*] Preparing nifti image for moving...
start ANTs
ANTs End time: 2024-07-11 13:36:38

In [10]:
dt_annotated = np.dtype([
    ('X(um)', 'f4'), ('Y(um)', 'f4'), ('Z(um)', 'f4'),
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
])

dt_registration = np.dtype([
    ('z', 'f4'), ('y', 'f4'), ('x', 'f4')
]) # 50 um scale

dt_annotation2 = np.dtype([
  ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
  ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
  ('SCA_x', 'f4'), ('SCA_y', 'f4'), ('SCA_z', 'f4'),
  ('atlas_id', 'u2')
])

cou3 = 0

for Sample in Samples:
    
    start_time = datetime.datetime.now()
    start_time2 = datetime.datetime.now()
    print("Start Start time:", start_time)

    print("### Read files ###")
    print(Sample)
    
    fw_dir_nu = f"/export3/Imaging/Abeta/{Sample}/"
    #print(fw_dir_nu)

    from concurrent.futures import ProcessPoolExecutor
    import concurrent.futures
    
    if 1==1:
    #try:
        grid_points_df = []
        transformed_points = []

        temp = np.fromfile(f"/export3/Imaging/Abeta/{Sample}/Transformed_to_R_atlas_space_Abeta_positive_voxel_coordinates.bin", dtype=dt_annotated)
        new_array = np.zeros((len(temp),), dtype=dt_annotation2)
        new_array["stitched_x"]  =  temp["X(um)"]
        new_array["stitched_y"]  =  temp["Y(um)"] 
        new_array["stitched_z"]  =  temp["Z(um)"] 
        new_array["mapped_z"]  =  temp["mapped_z"] 
        new_array["mapped_y"]  =  temp["mapped_y"] 
        new_array["mapped_x"]  =  temp["mapped_x"] 

        grid_points_df =  np.zeros((len(new_array),), dtype=dt_registration)

        transformed_points =  pd.DataFrame(transformed_points)
        grid_points_df =  pd.DataFrame(grid_points_df)
        grid_points_df["z"]  =  new_array["mapped_z"] * (1/50)
        grid_points_df["y"]  = new_array["mapped_y"]  * (1/50)
        grid_points_df["x"]  = new_array["mapped_x"]  * (1/50)
        start_time = datetime.datetime.now()
        print("Affine trasform Start time:", start_time)

        transformed_points = ants.apply_transforms_to_points(
            dim=3,
            points=grid_points_df,
            transformlist=["/export2/Imaging/ds4_Data4/atlas_R_test/slice_0_88_inv.mat"],
            whichtoinvert = [True]
        )

        end_time = datetime.datetime.now()
        print("Affine trasform End time:", end_time)
        print("Duration:", end_time - start_time)

        transformed_points = pd.DataFrame({'id': range(len(transformed_points)), 'x': transformed_points["x"], 'y': transformed_points["y"], 'z': transformed_points["z"]})


        print(len(np.unique(transformed_points["id"])))


        # Masking

        dt_mask = np.dtype([
            ('X(um)', 'f4'), ('Y(um)', 'f4'), ('Z(um)', 'f4'), ('atlasID', 'u2')
        ])

        mask_a =  np.fromfile("/export2/Imaging/ds4_Data4/atlas_R_test/CUBIC-space-grid-10um-resolution-transformed_with_annotations_modified_integrated_rounded_final.bin",  dtype=dt_mask)
        mask_a =  pd.DataFrame(mask_a)
        mask_a["x"] = mask_a["X(um)"] * (1/50)
        mask_a["y"] = mask_a["Y(um)"] * (1/50)
        mask_a["z"] = mask_a["Z(um)"] * (1/50)

        print(len(mask_a))
        end_time = datetime.datetime.now()
        print("Mask Read End time:", end_time)
        print("Duration:", end_time - start_time)

        import numpy as np
        import datetime
        from scipy.spatial import cKDTree

        def split_dataframe(df, n_splits=10):
            # Split the DataFrame into n_splits Equal Parts
            chunk_size = len(df) // n_splits
            return [df[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        # Split the mask into 10 equal parts
        splits = split_dataframe(mask_a[['x', 'y', 'z']], n_splits=10)

        grid_points = grid_points_df[['x', 'y', 'z']].values

        # Arrays to store search results for all splits
        all_distances = np.full(grid_points.shape[0], np.inf)
        all_indexes = np.full(grid_points.shape[0], -1)

        cou = 0
        for split in splits:
            print(cou)
            
            tree = cKDTree(split.values)

            distances, indexes = tree.query(grid_points, k=1, distance_upper_bound=3, n_jobs=-1)

            # Update the result if a shorter distance is found
            better_mask = distances < all_distances
            all_distances[better_mask] = distances[better_mask]
            all_indexes[better_mask] = indexes[better_mask]
            cou += 1

            del tree, distances, indexes, better_mask
            gc.collect()

            end_time = datetime.datetime.now()
            print(str(cou) + "_End time:", end_time)
            print("Duration:", end_time - start_time)

        # Extract points within the shortest distance of the square root of 3
        mask_manual = all_distances <= (3 ** 0.5) * (10/25) # Adjusted to √3 at 10 μm scale (originally √3 at 25 μm scale)

        del all_distances, all_indexes, mask_a
        gc.collect()

        print(np.sum(mask_manual))

        end_time = datetime.datetime.now()
        print("All End time:", end_time)
        print("Duration:", end_time - start_time)

        print("###############  SCA registration   #####################")

        import gc
        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        mask1 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas_R_test/cortex_grid.tif')
        mask1 [mask1 > 0] = 1.0 

        coordinates4 = np.argwhere(mask1 == 1)

        coordinates_df4 = pd.DataFrame(coordinates4, columns=['z', 'y', 'x'])

        mask3 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas_R_test/shono_grid.tif')
        mask3 [mask3 > 0] = 1.0 


        coordinates5 = np.argwhere(mask3 == 1)

        coordinates_df5 = pd.DataFrame(coordinates5, columns=['z', 'y', 'x'])

        from scipy.spatial import cKDTree

        transformed_points = transformed_points.reset_index(drop=True)


        tree = cKDTree(coordinates_df4[['x', 'y', 'z']].values)

        distances, indexes = tree.query(transformed_points[['x', 'y', 'z']].values, k=1, distance_upper_bound = 3, n_jobs = -1)

        mask_cortex = distances <=  1.4143
        mask_1 = transformed_points["y"] < 420

        from scipy.spatial import cKDTree

        transformed_points = transformed_points.reset_index(drop=True)

        tree = cKDTree(coordinates_df5[['x', 'y', 'z']].values)
        distances, indexes = tree.query(transformed_points[['x', 'y', 'z']].values, k=1, distance_upper_bound = 3, n_jobs = -1)

        mask_shono = distances <= 1.4143
        mask_2 = transformed_points["y"] >= 420 

        transformed_points_cortex = transformed_points[(mask_cortex) | (~mask_cortex & ~mask_shono & mask_1)]
        transformed_points_nokan_shono = transformed_points[(mask_shono) | (~mask_cortex & ~mask_shono & mask_2)]
        transformed_points_cortex_only = transformed_points[(mask_cortex & ~mask_shono) | (~mask_cortex & ~mask_shono & mask_1)]
        transformed_points_shono_only = transformed_points[(~mask_cortex & mask_shono) | (~mask_cortex & ~mask_shono & mask_2)]

        del distances, indexes,mask1, mask3, mask_cortex, mask_shono, mask_1, mask_2, coordinates5, coordinates_df5, tree, coordinates4, coordinates_df4
        gc.collect()

        end_time = datetime.datetime.now()
        print("End time4:", end_time)
        print("Duration:", end_time - start_time)

        
        # Concatenate the four parts while applying weights

        import pandas as pd
        import numpy as np
        import os
        import re
        import ants
        import datetime
        from concurrent.futures import ThreadPoolExecutor, as_completed
        import gc

        print("######## weight calculate #############")

        def compute_weight(y, left_overlap_start, left_overlap_end, right_overlap_start, right_overlap_end, left_min_weight, right_min_weight):
            # Initialize all weights to 1
            weight = np.ones_like(y)

            # Compute Gaussian weights for the left overlap
            left_indices = (y >= left_overlap_start) & (y <= left_overlap_end)
            if np.any(left_indices):
                left_sigma = (left_overlap_end - left_overlap_start) / 5
                left_center = left_overlap_end  # Set the center to the end of the left overlap
                weight[left_indices] = np.exp(-((y[left_indices] - left_center) ** 2) / (2 * left_sigma ** 2))
                # Scale the weights for the left overlap
                weight[left_indices] = left_min_weight + (1 - left_min_weight) * (weight[left_indices] - np.min(weight[left_indices])) / (np.max(weight[left_indices]) - np.min(weight[left_indices]))

            # Compute Gaussian weights for the right overlap
            right_indices = (y >= right_overlap_start) & (y <= right_overlap_end)
            if np.any(right_indices):
                right_sigma = (right_overlap_end - right_overlap_start) / 5
                right_center = right_overlap_start  # Set the center to the start of the right overlap
                weight[right_indices] = np.exp(-((y[right_indices] - right_center) ** 2) / (2 * right_sigma ** 2))
                # Scale the weights for the right overlap
                weight[right_indices] = right_min_weight + (1 - right_min_weight) * (weight[right_indices] - np.min(weight[right_indices])) / (np.max(weight[right_indices]) - np.min(weight[right_indices]))

            return weight
        
        def process_slice(i, transformed_points1, path, slice_num, additional_cortex, additional_shono): 


            start_slice, end_slice = slice_num

            print("slice: " +str(i)+  ", start:" + str(start_slice) + ", end: " + str(end_slice))

            if start_slice == 0 and end_slice != 528:  # start
                print("start edge: add minus y points")
                filtered_indices3 = transformed_points1["y"] < end_slice

            elif end_slice == 528 and start_slice != 0:  # end
                print("end edge: add over y points")
                filtered_indices3 = transformed_points1["y"] >= start_slice

            elif start_slice == 0 and end_slice == 528:  # all
                print("both edge: add over y points")
                filtered_indices3 = pd.Series([True] * len(transformed_points1))

            else:  # 通常の範囲
                filtered_indices3 = (transformed_points1["y"] >= start_slice) & (transformed_points1["y"] < end_slice)


            # Convert row and column indices to a DataFrame.
            # Here, y and z are intentionally swapped.
            extracted_points_df2 = []
            extracted_points_df2 = pd.DataFrame({'x': transformed_points1["x"][filtered_indices3], 
                                           'y':  transformed_points1["z"][filtered_indices3], 
                                           'z': transformed_points1["y"][filtered_indices3]-start_slice})


            temp_transformed_points2 = ants.apply_transforms_to_points(
            dim=3,
            points=extracted_points_df2,
            transformlist=[path + "slice_"+ str(start_slice) + "_"+ str(end_slice) +"_fwd.mat",
                           path + "slice_" + str(start_slice)+"_"+ str(end_slice) +"_inv.nii.gz"
                           ],
            whichtoinvert = [True, False]
            )

            del extracted_points_df2
            gc.collect()

            end_time = datetime.datetime.now()
            print("2nd transfer " + str(i) + "_end:", end_time)
            print("Duration:", end_time - start_time)

            temp_transformed_points2 = pd.DataFrame({'id': transformed_points1["id"][filtered_indices3],'ori_y': transformed_points1["y"][filtered_indices3], 'x': temp_transformed_points2["x"], 'y': temp_transformed_points2["z"] + start_slice, 'z': temp_transformed_points2["y"]})
            del filtered_indices3
            gc.collect()

            print("start " + str(i) + "_" + str(len(temp_transformed_points2)))

            alpha = 0.02
            if i == 0:
                temp_transformed_points2['weight'] = compute_weight(temp_transformed_points2['y'], 
                                                                       0, 10, 100, 220, alpha, alpha)  
                temp_transformed_points2 = temp_transformed_points2[temp_transformed_points2 ["y"] < 220]

            if i == 1:
                temp_transformed_points2['weight'] = compute_weight(temp_transformed_points2['y'], 
                                                                       100, 220, 270, 350 , alpha, alpha)
                temp_transformed_points2 = temp_transformed_points2[(temp_transformed_points2["y"] >= 100)
                                                & (temp_transformed_points2["y"] < 350)]

            if i == 2:
                temp_transformed_points2['weight'] = compute_weight(temp_transformed_points2['y'], 
                                                                       270, 350, 360, 450, alpha,alpha) 
                temp_transformed_points2 = temp_transformed_points2[((temp_transformed_points2["y"] >= 270)
                                                & (temp_transformed_points2["y"] < 450))]

                extracted_points_df2 = []
                extracted_points_df2 = pd.DataFrame({'x': additional_cortex["x"], 
                                               'y':  additional_cortex["z"], 
                                               'z': additional_cortex["y"] - start_slice})


                temp_additional_cortex2 = ants.apply_transforms_to_points(
                dim=3,
                points=extracted_points_df2,
                transformlist=[path + "slice_"+ str(start_slice) + "_"+ str(end_slice) +"_fwd.mat",
                               path + "slice_" + str(start_slice)+"_"+ str(end_slice) +"_inv.nii.gz"
                               ],
                whichtoinvert = [True, False]
                )

                temp_additional_cortex2 = pd.DataFrame({'id': additional_cortex["id"],'ori_y': additional_cortex["y"], 'x': temp_additional_cortex2["x"], 'y': temp_additional_cortex2["z"] + start_slice, 'z': temp_additional_cortex2["y"]})

                temp_additional_cortex2 = temp_additional_cortex2[temp_additional_cortex2['ori_y']>= 444]

                temp_additional_cortex2['weight'] = np.ones_like(len(temp_additional_cortex2))
                temp_transformed_points2 = pd.concat([temp_transformed_points2, temp_additional_cortex2], ignore_index=True)

                del temp_additional_cortex2
                gc.collect()

            if i == 3:
                temp_transformed_points2['weight'] = compute_weight(temp_transformed_points2['y'], 
                                                                      360, 450, 518, 528, alpha, alpha)
                temp_transformed_points2 = temp_transformed_points2[(temp_transformed_points2["y"] >= 360)]

                extracted_points_df2 = []
                extracted_points_df2 = pd.DataFrame({'x': additional_shono["x"], 
                                               'y':  additional_shono["z"], 
                                               'z': additional_shono["y"] - start_slice})


                temp_additional_shono2 = ants.apply_transforms_to_points(
                dim=3,
                points=extracted_points_df2,
                transformlist=[path + "slice_"+ str(start_slice) + "_"+ str(end_slice) +"_fwd.mat",
                               path + "slice_" + str(start_slice)+"_"+ str(end_slice) +"_inv.nii.gz"
                               ],
                whichtoinvert = [True, False]
                )

                temp_additional_shono2 = pd.DataFrame({'id': additional_shono["id"],'ori_y': additional_shono["y"], 'x': temp_additional_shono2["x"], 'y': temp_additional_shono2["z"] + start_slice, 'z': temp_additional_shono2["y"]})

                temp_additional_shono2= temp_additional_shono2[temp_additional_shono2['ori_y'] < 387]

                temp_additional_shono2['weight'] = np.ones_like(len(temp_additional_shono2))
                temp_transformed_points2 = pd.concat([temp_transformed_points2, temp_additional_shono2], ignore_index=True)

                del temp_additional_shono2
                gc.collect()



            print("end " + str(i) + "_" +str(len(temp_transformed_points2)))

            return temp_transformed_points2


        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        # Provide the source point clouds and registration equations for the four registrations.
        # cortex = cortex
        # nokan (Japanese) = brain stem
        # shono (Japanese) = cerebellum

        transformed_points_cortex = transformed_points_cortex.reset_index(drop=True)
        transformed_points_nokan_shono = transformed_points_nokan_shono.reset_index(drop=True)
        transformed_points_cortex_only = transformed_points_cortex_only.reset_index(drop=True)
        transformed_points_shono_only = transformed_points_shono_only.reset_index(drop=True)

        srcs = [transformed_points_cortex, transformed_points_cortex, transformed_points_cortex,  transformed_points_nokan_shono]
        additional_cortex = transformed_points_cortex_only
        additional_shono = transformed_points_shono_only
        main_path = "/export2/Imaging/ds4_Data4/atlas_R_test/"
        paths = [main_path + "3D_block_registerded_cortex_inv_fwd/", main_path + "3D_block_registerded_cortex_inv_fwd/", main_path + "3D_block_registerded_cortex_inv_fwd/",
               main_path + "3D_block_registerded_nokan_shono_inv_fwd/"]

        slice_nums = [(0, 250), (0, 528), (250, 528), (250, 528)]

        results = []

        df = pd.DataFrame()  

        with ThreadPoolExecutor(max_workers=4) as executor:
            futures4 = {executor.submit(process_slice, i,  srcs[i], paths[i], slice_nums[i], additional_cortex, additional_shono): i for i in range(len(srcs))}

            for future in as_completed(futures4):
                index = futures4[future]
                try:
                    result = future.result()
                    results.append((index, result))  # Save the index and results as a tuple
                except Exception as e:
                    print(f"Error processing slice: {index} with error {e}")

        # Sort by index and then merge the results into the DataFrame.
        # This ensures that the results are in the correct order of i.

        if 1== 1: 
            results.sort(key=lambda x: x[0])  # Sort the results by index
            for _, result in results:
                df = pd.concat([df, result], ignore_index=True)

            del results, futures4
            gc.collect()

        end_time = datetime.datetime.now()
        print("End time:", end_time)
        print("Duration:", end_time - start_time)

        print("######## integrate #############")

        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        # Calculate the total weight for each ID
        weight_sum = df.groupby('id')['weight'].transform('sum')

        end_time = datetime.datetime.now()
        print("End time1:", end_time)
        print("Duration:", end_time - start_time)

        # Normalize the weights
        df['normalized_weight'] = df['weight'] / weight_sum

        del weight_sum
        gc.collect()

        end_time = datetime.datetime.now()
        print("End time2:", end_time)
        print("Duration:", end_time - start_time)

        # Multiply weights with x, y, and z to calculate weighted values
        df['x'] = df['normalized_weight'] * df['x']
        df['y'] = df['normalized_weight'] * df['y']
        df['z'] = df['normalized_weight'] * df['z']

        end_time = datetime.datetime.now()
        print("End time3:", end_time)
        print("Duration:", end_time - start_time)

        # Calculate the sum of x, y, and z for each ID
        averaged_coordinates = df.groupby('id').agg(
            x=('x', 'sum'),
            y=('y', 'sum'),
            z=('z', 'sum')
        ).reset_index()

        end_time = datetime.datetime.now()
        print("End time4:", end_time)
        print("Duration:", end_time - start_time)

        # Apply an additional affine transformation

        src_averaged_coordinates = averaged_coordinates.copy()

        averaged_coordinates = ants.apply_transforms_to_points(
            dim=3,
            points=src_averaged_coordinates,
            transformlist=["/export2/Imaging/ds4_Data4/atlas_R_test/3D_block_registerded_cortex_SCA_for_SCN_registration_ds4_inv_fwd/slice_0_528_inv_Affine_horizontal.mat"],
            whichtoinvert = [True]
        )

        del src_averaged_coordinates
        gc.collect()

        end_time = datetime.datetime.now()
        print("Affine End time4.5:", end_time)
        print("Duration:", end_time - start_time)

        if 1==1:
            scale = 1/50

            depth_ori = 320 
            height_ori = 528
            width_ori = 456

            print(depth_ori)
            print(height_ori)
            print(width_ori)
            
            # SCA = Spatial Cellome Atlas = Neuron Atlas

            img_filename_Nuclear_Isocortex_to_ori = f"/export3/Imaging/Abeta/{Sample}/Abeta_50um_scale_on_SCA.tif"

            img_N_ori,_ = np.histogramdd(
                np.vstack([
                    averaged_coordinates["z"],
                    averaged_coordinates["y"], 
                    averaged_coordinates["x"] 
                    ]).T,
                    bins=(depth_ori, height_ori, width_ori),
                    range=[(0,depth_ori),(0,height_ori),(0,width_ori)]
                )

            tifffile.imsave(
                    img_filename_Nuclear_Isocortex_to_ori,
                    img_N_ori.astype(np.float32)
                )

            del img_N_ori
            gc.collect()

        end_time = datetime.datetime.now()
        print("End time5:", end_time)
        print("Duration:", end_time - start_time)


        print(len(np.unique(df["id"])))
        print(len(np.unique(transformed_points["id"])))

        print("##############  annotation ############")

        ## Code for Adding Atlas Annotations

        start_time = datetime.datetime.now()
        print("Annotation Start time:", start_time)

        annotation_10 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas/annotation_10_LUT_modified_final.tif') # 545 → 869

        # Extract coordinates with non-zero values and their corresponding values.
        # These values represent atlas annotation IDs (aligned with Allen Brain Atlas CCFv3).
        non_zero_coords = np.argwhere(annotation_10)
        non_zero_values = annotation_10[non_zero_coords[:, 0], non_zero_coords[:, 1], non_zero_coords[:, 2]]

        df_non_zero = pd.DataFrame(non_zero_coords, columns=['z', 'y', 'x'])
        df_non_zero['id'] = np.round(non_zero_values)

        df_non_zero['z'] = (np.round(df_non_zero['z']) + 0.5) * (10/25) # Keep the value as 10 and add 0.5 to ensure it aligns with the voxel center.
        df_non_zero['y'] = (np.round(df_non_zero['y']) + 0.5) * (10/25)
        df_non_zero['x'] = (np.round(df_non_zero['x']) + 0.5) * (10/25)

        del  non_zero_coords, non_zero_values
        gc.collect()

        end_time = datetime.datetime.now()
        print("End time1:", end_time)
        print("Duration:", end_time - start_time)

        start_time = datetime.datetime.now()
        print("Tree Start time:", start_time)


        def split_dataframe(df, n_splits=10):
            """Split the DataFrame into n_splits equal parts."""
            chunk_size = len(df) // n_splits
            return [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]


        splits = split_dataframe(df_non_zero[['x', 'y', 'z']], n_splits=10)

        averaged_coords = averaged_coordinates[['x', 'y', 'z']].values

        all_distances = np.full(averaged_coords.shape[0], np.inf)
        all_indexes = np.full(averaged_coords.shape[0], -1)

        cou = 0
        for split in splits:
            print(cou)
            tree = cKDTree(split.values)
            distances, indexes = tree.query(averaged_coords, k=1, distance_upper_bound=2, n_jobs=-1)

            mask = distances < all_distances
            all_distances[mask] = distances[mask]
            all_indexes[mask] = indexes[mask] + split.index.start  # Add an offset to the index
            cou += 1
            del tree, distances, indexes, mask
            gc.collect()

            end_time = datetime.datetime.now()
            print(str(cou) + " End time:", end_time)
            print("Duration:", end_time - start_time)

        # Update the nearest_id if the calculated distance is less than the specified threshold
        distance_threshold = (3 ** 0.5) * (10 / 25)   # Adjusted from √2 at 25 μm scale to √3 at 10 μm scale
        mask = all_distances <= distance_threshold

        averaged_coordinates['nearest_id'] = 0  
        averaged_coordinates.loc[mask, 'nearest_id'] = df_non_zero.iloc[all_indexes[mask]]['id'].values

        depth, height, width = annotation_10.shape

        # df_saveのz, y, x座標を整数に丸める
        z_indices = (averaged_coordinates['z']*(25/10)).astype(int)
        y_indices = (averaged_coordinates['y']*(25/10)).astype(int)
        x_indices = (averaged_coordinates['x']*(25/10)).astype(int)

        # Verify whether the coordinates fall within the valid range of annotation_10
        in_range_mask = (z_indices >= 0) & (z_indices < depth) & (y_indices >= 0) & (y_indices < height) & (x_indices >= 0) & (x_indices < width)

        # Use a mask for indices within the range to extract values from annotation_10
        extracted_values = annotation_10[z_indices[in_range_mask], y_indices[in_range_mask], x_indices[in_range_mask]]


        valid_values_mask = np.zeros(len(averaged_coordinates), dtype=bool)
        valid_values_mask[in_range_mask] = extracted_values > 0
        
        averaged_coordinates['nearest_id'][~valid_values_mask] = 0

        del annotation_10, z_indices, y_indices, x_indices, in_range_mask, valid_values_mask, extracted_values
        gc.collect()

        new_array ['SCA_x'] = averaged_coordinates ["x"] * 25
        new_array ['SCA_y'] = averaged_coordinates ["y"] * 25
        new_array ['SCA_z'] = averaged_coordinates ["z"] * 25
        new_array ['atlas_id'][mask_manual] = averaged_coordinates ["nearest_id"][mask_manual]


        new_array.tofile(fw_dir_nu + "/SCA_registered_cell_coordinates_ver2_2.bin")


        del all_distances, all_indexes, mask, averaged_coords, splits, new_array
        gc.collect()

        end_time = datetime.datetime.now()
        print("Averaged cell coordinates Save End time:", end_time)
        print("Duration:", end_time - start_time)

        print(averaged_coordinates["nearest_id"].max())


        start_time = datetime.datetime.now()
        print("Start time:", start_time)

        tree = []

        averaged_coordinates_part = averaged_coordinates [averaged_coordinates["nearest_id"]>0]
        tree = cKDTree(averaged_coordinates_part[['x', 'y', 'z']].values)

        end_time = datetime.datetime.now()
        print("End time1:", end_time)
        print("Duration:", end_time - start_time)

        # Calculate the shortest distance from each point in averaged_coordinates to the points in df_non_zero
        distances2, indexes2 = tree.query(df_non_zero[['x', 'y', 'z']].values, k=1, distance_upper_bound=2, n_jobs=-1)

        del tree, averaged_coordinates_part
        gc.collect()

        end_time = datetime.datetime.now()
        print("All End time2", end_time)
        print("Duration:", end_time - start_time)


        print("#############  calculate region volume   ################")

        data_GO = pd.read_csv(filepath_or_buffer="/export3/Imaging/atlas_for_fig/GOvsID_6region_removed.csv", encoding="ms932", sep=",")
        
        ids = data_GO["id"].astype(int)
        ids=np.array(ids)


        mask = distances2 <= 2

        depth_ori = 800 
        height_ori = 1320
        width_ori = 1140

        annotation_10 = tifffile.imread('/export2/Imaging/ds4_Data4/atlas/annotation_10_LUT_modified_final.tif') 

        non_zero_coords = np.argwhere(annotation_10)
        non_zero_values = annotation_10[non_zero_coords[:, 0], non_zero_coords[:, 1], non_zero_coords[:, 2]]

        df_non_zero = pd.DataFrame(non_zero_coords, columns=['z', 'y', 'x'])
        df_non_zero['id'] = np.round(non_zero_values)

        df_non_zero['z'] = np.round(df_non_zero['z']) 
        df_non_zero['y'] = np.round(df_non_zero['y']) 
        df_non_zero['x'] = np.round(df_non_zero['x']) 

        del annotation_10, non_zero_coords, non_zero_values
        gc.collect()

        z_indices = df_non_zero['z'][mask].values.astype(int)
        y_indices = df_non_zero['y'][mask].values.astype(int)
        x_indices = df_non_zero['x'][mask].values.astype(int)
        annotation_10_ids = df_non_zero['id'][mask].values.astype(np.uint16)

        # For verification: Match the data type of the 3D array with ids
        image_3d = np.zeros((depth_ori, height_ori, width_ori), dtype=np.uint16)
        image_3d[z_indices, y_indices, x_indices] = annotation_10_ids
        tifffile.imwrite(fw_dir_nu +  "/annotated_annotation_10_2.tif" , image_3d)


        del z_indices, y_indices, x_indices, annotation_10_ids, image_3d
        gc.collect()

        end_time = datetime.datetime.now()
        print("Tiff save End time1:", end_time)
        print("Duration:", end_time - start_time)

        # Count the occurrences of each ID in the id column after applying the mask
        id_counts = df_non_zero['id'][mask].value_counts()

        # Filter the aggregated results based on the unique ID list (treat non-existent IDs as 0)
        filtered_counts = id_counts.reindex(ids, fill_value=0)

        df_filtered = filtered_counts.reset_index()
        df_filtered.columns = ['atlasID', 'iso_10um_voxel_count']  

        csv_file_path =  fw_dir_nu +  "/Regional_iso_10um_voxel_count2.csv"  

        df_filtered.to_csv(csv_file_path, index=False)

        del distances2, indexes2, df_non_zero, transformed_points, grid_points_df, averaged_coordinates, mask_manual
        gc.collect()

        cou3 += 1
    #except: print("skip")

Start Start time: 2024-07-11 20:16:01.487220
### Read files ###
#4_APPmodel_Abeta_Ctr3m_1
Affine trasform Start time: 2024-07-11 20:16:04.487534
Affine trasform End time: 2024-07-11 20:16:04.584230
Duration: 0:00:00.096696
188354
1113543350
Mask Read End time: 2024-07-11 20:18:39.987496
Duration: 0:02:35.499962
Start time: 2024-07-11 20:18:39.989971
0
1_End time: 2024-07-11 20:21:01.494946
Duration: 0:02:21.504975
1
2_End time: 2024-07-11 20:22:32.876593
Duration: 0:03:52.886622
2
3_End time: 2024-07-11 20:24:06.103046
Duration: 0:05:26.113075
3
4_End time: 2024-07-11 20:25:36.444249
Duration: 0:06:56.454278
4
5_End time: 2024-07-11 20:27:07.977853
Duration: 0:08:27.987882
5
6_End time: 2024-07-11 20:28:36.857253
Duration: 0:09:56.867282
6
7_End time: 2024-07-11 20:30:07.156347
Duration: 0:11:27.166376
7
8_End time: 2024-07-11 20:31:33.291494
Duration: 0:12:53.301523
8
9_End time: 2024-07-11 20:32:59.129522
Duration: 0:14:19.139551
9
10_End time: 2024-07-11 20:34:22.889604
Duration: 0:

6_End time: 2024-07-11 21:19:50.953986
Duration: 0:08:44.239692
6
7_End time: 2024-07-11 21:21:07.212895
Duration: 0:10:00.498601
7
8_End time: 2024-07-11 21:22:23.430376
Duration: 0:11:16.716082
8
9_End time: 2024-07-11 21:23:42.283790
Duration: 0:12:35.569496
9
10_End time: 2024-07-11 21:25:01.481340
Duration: 0:13:54.767046
108986
All End time: 2024-07-11 21:25:03.587976
Duration: 0:13:56.873682
###############  SCA registration   #####################
Start time: 2024-07-11 21:25:03.588103
End time4: 2024-07-11 21:26:04.276180
Duration: 0:01:00.688077
######## weight calculate #############
Start time: 2024-07-11 21:26:04.276698
slice: 0, start:0, end: 250slice: 1, start:0, end: 528
both edge: add over y points

start edge: add minus y points
slice: 2, start:250, end: 528
end edge: add over y points
slice: 3, start:250, end: 528
end edge: add over y points
2nd transfer 0_end: 2024-07-11 21:26:09.008723
Duration: 0:00:04.732025
2nd transfer 3_end:2nd transfer 2_end: 2024-07-11 21:26

2nd transfer 0_end: 2024-07-11 22:18:31.998776
Duration: 0:00:08.253511
2nd transfer 3_end: 2024-07-11 22:18:44.475201
Duration: 0:00:20.729936
2nd transfer 2_end: 2024-07-11 22:18:44.6225662nd transfer 1_end: 2024-07-11 22:18:44.760985
Duration: 0:00:21.015720
Duration: 0:00:20.877301

start 0_9309
start 3_14943
start 2_39722
start 1_49031
end 0_6326
end 1_31343
end 3_14988
end 2_35299
End time: 2024-07-11 22:18:53.411044
Duration: 0:00:29.665779
######## integrate #############
Start time: 2024-07-11 22:18:53.411222
End time1: 2024-07-11 22:18:53.424109
Duration: 0:00:00.012887
End time2: 2024-07-11 22:18:53.530634
Duration: 0:00:00.119412
End time3: 2024-07-11 22:18:53.535135
Duration: 0:00:00.123913
End time4: 2024-07-11 22:18:53.582124
Duration: 0:00:00.170902
Affine End time4.5: 2024-07-11 22:18:53.704440
Duration: 0:00:00.293218
320
528
456
End time5: 2024-07-11 22:18:54.427714
Duration: 0:00:01.016492
54466
54466
##############  annotation ############
Annotation Start time: 20

End time4: 2024-07-11 23:11:29.027272
Duration: 0:00:06.135318
Affine End time4.5: 2024-07-11 23:11:30.035350
Duration: 0:00:07.143396
320
528
456
End time5: 2024-07-11 23:11:31.254126
Duration: 0:00:08.362172
5986388
5986388
##############  annotation ############
Annotation Start time: 2024-07-11 23:11:31.675573
End time1: 2024-07-11 23:12:23.614729
Duration: 0:00:51.939156
Tree Start time: 2024-07-11 23:12:23.614915
0
1 End time: 2024-07-11 23:13:13.050553
Duration: 0:00:49.435638
1
2 End time: 2024-07-11 23:13:44.767526
Duration: 0:01:21.152611
2
3 End time: 2024-07-11 23:14:19.458718
Duration: 0:01:55.843803
3
4 End time: 2024-07-11 23:14:56.392086
Duration: 0:02:32.777171
4
5 End time: 2024-07-11 23:15:29.819204
Duration: 0:03:06.204289
5
6 End time: 2024-07-11 23:16:02.657173
Duration: 0:03:39.042258
6
7 End time: 2024-07-11 23:16:36.047041
Duration: 0:04:12.432126
7
8 End time: 2024-07-11 23:17:08.434463
Duration: 0:04:44.819548
8
9 End time: 2024-07-11 23:17:41.597269
Duration

5 End time: 2024-07-12 00:10:38.315701
Duration: 0:03:09.599944
5
6 End time: 2024-07-12 00:11:12.409511
Duration: 0:03:43.693754
6
7 End time: 2024-07-12 00:11:46.713895
Duration: 0:04:17.998138
7
8 End time: 2024-07-12 00:12:19.812064
Duration: 0:04:51.096307
8
9 End time: 2024-07-12 00:12:54.060157
Duration: 0:05:25.344400
9
10 End time: 2024-07-12 00:13:28.566147
Duration: 0:05:59.850390
Averaged cell coordinates Save End time: 2024-07-12 00:13:33.183666
Duration: 0:06:04.467909
61277
Start time: 2024-07-12 00:13:33.195449
End time1: 2024-07-12 00:13:38.488893
Duration: 0:00:05.293444
All End time2 2024-07-12 00:14:25.261902
Duration: 0:00:52.066453
#############  calculate region volume   ################
Tiff save End time1: 2024-07-12 00:15:55.280498
Duration: 0:02:22.085049
